### Import Libraries

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

### Connect to SQLite3

In [2]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("sports_performance.db")
cursor = conn.cursor()

### Creation of Players Table

In [3]:
# Creating Players table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Players (
    player_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER CHECK(age BETWEEN 18 AND 40),
    nationality TEXT NOT NULL,
    ranking INTEGER NULL CHECK(ranking > 0),
    gender TEXT CHECK(gender IN ('Male', 'Female')),
    experience_level TEXT CHECK(experience_level IN ('Beginner', 'Intermediate', 'Professional'))
);
''')

### Creation of Tournaments Table

In [4]:
# Creating Tournaments table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Tournaments (
    tournament_id INTEGER PRIMARY KEY AUTOINCREMENT,
    tournament_name TEXT NOT NULL,
    location TEXT NOT NULL,
    year INTEGER CHECK(year BETWEEN 2000 AND 2025),
    category TEXT CHECK(category IN ('Regional', 'National', 'International'))
);
''')

### Creation of Matches Table

In [5]:
# Creating Matches table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Matches (
    match_id INTEGER PRIMARY KEY AUTOINCREMENT,
    tournament_id INTEGER,
    player1_id INTEGER,
    player2_id INTEGER,
    round TEXT CHECK(round IN ('Group Stage', 'Quarterfinal', 'Semifinal', 'Final')),
    score TEXT NULL,
    match_date DATE NOT NULL,
    FOREIGN KEY (tournament_id) REFERENCES Tournaments(tournament_id),
    FOREIGN KEY (player1_id) REFERENCES Players(player_id),
    FOREIGN KEY (player2_id) REFERENCES Players(player_id)
);
''')

### Creation of Performance Table

In [6]:
# Creating Performance table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Performance (
    performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    match_id INTEGER,
    points_scored INTEGER CHECK(points_scored >= 0),
    fouls INTEGER CHECK(fouls >= 0),
    ranking_change INTEGER,
    FOREIGN KEY (player_id) REFERENCES Players(player_id),
    FOREIGN KEY (match_id) REFERENCES Matches(match_id)
);
''')

### Creation of Sponsorships Table

In [7]:
# Creating Sponsorships table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Sponsorships (
    sponsor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sponsor_name TEXT NOT NULL,
    player_id INTEGER,
    contract_value REAL CHECK(contract_value > 0),
    duration_years INTEGER CHECK(duration_years > 0),
    FOREIGN KEY (player_id) REFERENCES Players(player_id)
);
''')

### Insert Hardcoded Data into the Tables (Players,Tournaments , Matches, Performance, Sponsorships)

In [8]:
# Inserting Data into Players Table
cursor.execute('''
  INSERT INTO Players (name, age, nationality, ranking, gender, experience_level) VALUES
  ('John Doe', 25, 'USA', 10, 'Male', 'Professional'),
  ('Emily Smith', 22, 'UK', 15, 'Female', 'Intermediate'),
  ('Rahul Sharma', 28, 'India', 8, 'Male', 'Professional'),
  ('Carlos Gomez', 30, 'Spain', 12, 'Male', 'Professional'),
  ('Wei Zhang', 24, 'China', 20, 'Female', 'Intermediate'),
  ('Michael Brown', 27, 'USA', 18, 'Male', 'Intermediate'),
  ('Sophia Wilson', 23, 'UK', 30, 'Female', 'Beginner'),
  ('Anil Kumar', 29, 'India', 40, 'Male', 'Professional'),
  ('Hiroshi Tanaka', 26, 'Japan', 22, 'Male', 'Intermediate'),
  ('Maria Sanchez', 21, 'Spain', 35, 'Female', 'Beginner');
  ''')
# Inserting Data into Tournaments Table
cursor.execute('''
  INSERT INTO Tournaments (tournament_name, location, year, category) VALUES
  ('World Championship', 'New York', 2024, 'International'),
  ('European Open', 'London', 2023, 'National'),
  ('Asia Cup', 'Beijing', 2022, 'Regional'),
  ('Americas Cup', 'Los Angeles', 2025, 'International'),
  ('African Championship', 'Cape Town', 2021, 'Regional');
  ''')
# -- Inserting Data into Matches Table
cursor.execute('''
  INSERT INTO Matches (tournament_id, player1_id, player2_id, round, score, match_date) VALUES
  (1, 1, 2, 'Final', '21-19, 18-21, 21-17', '2024-03-01'),
  (2, 3, 4, 'Semifinal', '21-15, 22-20', '2023-06-15'),
  (3, 2, 5, 'Quarterfinal', '21-18, 21-19', '2022-08-10'),
  (4, 6, 7, 'Group Stage', '21-14, 21-16', '2025-02-10'),
  (5, 8, 9, 'Final', '22-20, 21-19', '2021-07-15');
  ''')
# -- Inserting Data into Performance Table
cursor.execute('''
  INSERT INTO Performance (player_id, match_id, points_scored, fouls, ranking_change) VALUES
  (1, 1, 60, 2, +2),
  (2, 1, 55, 3, -1),
  (3, 2, 45, 1, +3),
  (4, 2, 48, 2, -2),
  (2, 3, 50, 1, +1),
  (6, 4, 30, 0, +4),
  (7, 4, 20, 1, -1),
  (8, 5, 40, 2, +3),
  (9, 5, 35, 3, -2);
  ''')
# Inserting Data into Sponsorships Table
cursor.execute('''
  INSERT INTO Sponsorships (sponsor_name, player_id, contract_value, duration_years) VALUES
  ('Nike', 1, 500000, 3),
  ('Adidas', 2, 450000, 2),
  ('Yonex', 3, 300000, 1),
  ('Li-Ning', 4, 350000, 2),
  ('Victor', 5, 400000, 3),
  ('Puma', 6, 250000, 2),
  ('Wilson', 7, 200000, 1),
  ('Babolat', 8, 275000, 3),
  ('Head', 9, 320000, 2);
  ''')

### Hardcoded Names that is inserted in the table and select random names

In [9]:
# Sample lists for names
first_names = ["John", "Alice", "Raj", "Maria", "Ans", "James", "Elena", "Amir", "Sofia", "Samran"]
middle_names = ["Michael", "Lee", "Singh", "Patel", "Ahmed", "William", "Marie", "Noel", "Khan", "Luis"]
last_names = ["Smith", "Liaqat", "Rana", "Lopez", "Riaz", "Johnson", "Fernandez", "Ali", "Garcia", "Malik"]

### Generte Random data as Required.

In [10]:
# Function to generate random data for the tables (Players,Tournaments , Matches, Performance, Sponsorships)
def generate_random_data(num_records, table):
    np.random.seed(42)
    if table == "Players":
        players = [
            (f"Player{i}", np.random.randint(18, 40), random.choice(["USA", "UK", "India", "Spain", "China"]),
             i + 1, random.choice(["Male", "Female"]), random.choice(["Beginner", "Intermediate", "Professional"]))
            for i in range(num_records)
        ]
        cursor.executemany("INSERT INTO Players (name, age, nationality, ranking, gender, experience_level) VALUES (?, ?, ?, ?, ?, ?);", players)
    elif table == "Tournaments":
        tournaments = [
            (f"Tournament{i}", random.choice(["New York", "London", "Beijing", "Tokyo", "Sydney"]), np.random.randint(2000, 2025), random.choice(["Regional", "National", "International"]))
            for i in range(num_records // 10)
        ]
        cursor.executemany("INSERT INTO Tournaments (tournament_name, location, year, category) VALUES (?, ?, ?, ?);", tournaments)
    elif table == "Matches":
        matches = [
            (random.randint(1, 100), random.randint(1, num_records), random.randint(1, num_records),
             random.choice(["Group Stage", "Quarterfinal", "Semifinal", "Final"]),
             f"{random.randint(10, 30)}-{random.randint(10, 30)}",
             f"202{random.randint(0, 3)}-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}")
            for _ in range(num_records)
        ]
        cursor.executemany("INSERT INTO Matches (tournament_id, player1_id, player2_id, round, score, match_date) VALUES (?, ?, ?, ?, ?, ?);", matches)
    elif table == "Performance":
        performance = [
            (random.randint(1, num_records), random.randint(1, num_records), random.randint(0, 100), random.randint(0, 10), random.randint(-5, 5))
            for _ in range(num_records)
        ]
        cursor.executemany("INSERT INTO Performance (player_id, match_id, points_scored, fouls, ranking_change) VALUES (?, ?, ?, ?, ?);", performance)
    elif table == "Sponsorships":
        sponsorships = [
            (random.choice(["Nike", "Adidas", "Yonex", "Li-Ning", "Victor"]), random.randint(1, num_records), random.uniform(50000, 500000), random.randint(1, 5))
            for _ in range(num_records // 5)
        ]
        cursor.executemany("INSERT INTO Sponsorships (sponsor_name, player_id, contract_value, duration_years) VALUES (?, ?, ?, ?);", sponsorships)

### Function Callling

In [11]:
# Generating Data for all tables
generate_random_data(1000, "Players")
generate_random_data(100, "Tournaments")
generate_random_data(300, "Matches")
generate_random_data(400, "Performance")
generate_random_data(50, "Sponsorships")

### Duplicate Player inserting into the Players table

In [12]:
# Introduce 50 duplicate player random records using loop
for _ in range(50):
    cursor.execute("SELECT name, age, nationality, ranking, gender, experience_level FROM Players ORDER BY RANDOM() LIMIT 1;")
    duplicate_player = cursor.fetchone()
    cursor.execute("INSERT INTO Players (name, age, nationality, ranking, gender, experience_level) VALUES (?, ?, ?, ?, ?, ?);", duplicate_player)

### Add null values in different tables

In [13]:
# Introduce missing rankings and experience levels in Players
cursor.execute("UPDATE Players SET ranking = NULL WHERE player_id % 5 = 0;")
cursor.execute("UPDATE Players SET experience_level = NULL WHERE player_id % 7 = 0;")

In [14]:
# Add matches with missing scores
cursor.execute("UPDATE Matches SET score = NULL WHERE match_id % 4 = 0;")

In [15]:
print("Database and tables created successfully with sample data!")

Database and tables created successfully with sample data!


### See Tables in the database

In [16]:
# Get list of all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display tables
print("Tables in the database:", tables)

Tables in the database: [('Players',), ('sqlite_sequence',), ('Tournaments',), ('Matches',), ('Performance',), ('Sponsorships',)]


### Delete Data from the database (just for testing)

In [57]:
# Optional just for testing

# Execute DELETE query
query = "DELETE FROM Players;"
# query = "DELETE FROM sqlite_sequence;"
# query = "DELETE FROM Tournaments;"
# query = "DELETE FROM Matches;"
# query = "DELETE FROM Performance;"
# query = "DELETE FROM Sponsorships;"


conn.execute(query)
conn.commit()

### Generic function to execute the query

In [17]:
def func_read_query(query):
    df = pd.read_sql(query, conn)
    return df

### Queries for the testing the Database

In [18]:
func_read_query("SELECT * FROM Performance Limit 10;")

,performance_id,player_id,match_id,points_scored,fouls,ranking_change
0,1,1,1,60,2,2
1,2,2,1,55,3,-1
2,3,3,2,45,1,3
3,4,4,2,48,2,-2
4,5,2,3,50,1,1
5,6,6,4,30,0,4
6,7,7,4,20,1,-1
7,8,8,5,40,2,3
8,9,9,5,35,3,-2
9,10,331,382,65,9,-3


In [19]:
func_read_query("SELECT * FROM Sponsorships Limit 5;")

,sponsor_id,sponsor_name,player_id,contract_value,duration_years
0,1,Nike,1,500000.0,3
1,2,Adidas,2,450000.0,2
2,3,Yonex,3,300000.0,1
3,4,Li-Ning,4,350000.0,2
4,5,Victor,5,400000.0,3


In [20]:
func_read_query("SELECT * FROM Matches Limit 5;")

,match_id,tournament_id,player1_id,player2_id,round,score,match_date
0,1,1,1,2,Final,"21-19, 18-21, 21-17",2024-03-01
1,2,2,3,4,Semifinal,"21-15, 22-20",2023-06-15
2,3,3,2,5,Quarterfinal,"21-18, 21-19",2022-08-10
3,4,4,6,7,Group Stage,None,2025-02-10
4,5,5,8,9,Final,"22-20, 21-19",2021-07-15


In [21]:
func_read_query("SELECT * FROM Tournaments;")

,tournament_id,tournament_name,location,year,category
0,1,World Championship,New York,2024,International
1,2,European Open,London,2023,National
2,3,Asia Cup,Beijing,2022,Regional
3,4,Americas Cup,Los Angeles,2025,International
4,5,African Championship,Cape Town,2021,Regional
5,6,Tournament0,Tokyo,2006,National
6,7,Tournament1,New York,2019,International
7,8,Tournament2,Sydney,2014,International
8,9,Tournament3,Tokyo,2010,Regional
9,10,Tournament4,Beijing,2007,National


In [22]:
func_read_query("SELECT * FROM Players LIMIT 10;")

,player_id,name,age,nationality,ranking,gender,experience_level
0,1,John Doe,25,USA,10.0,Male,Professional
1,2,Emily Smith,22,UK,15.0,Female,Intermediate
2,3,Rahul Sharma,28,India,8.0,Male,Professional
3,4,Carlos Gomez,30,Spain,12.0,Male,Professional
4,5,Wei Zhang,24,China,NaN,Female,Intermediate
5,6,Michael Brown,27,USA,18.0,Male,Intermediate
6,7,Sophia Wilson,23,UK,30.0,Female,None
7,8,Anil Kumar,29,India,40.0,Male,Professional
8,9,Hiroshi Tanaka,26,Japan,22.0,Male,Intermediate
9,10,Maria Sanchez,21,Spain,NaN,Female,Beginner


In [23]:
func_read_query("SELECT * FROM Matches Limit 5;")

,match_id,tournament_id,player1_id,player2_id,round,score,match_date
0,1,1,1,2,Final,"21-19, 18-21, 21-17",2024-03-01
1,2,2,3,4,Semifinal,"21-15, 22-20",2023-06-15
2,3,3,2,5,Quarterfinal,"21-18, 21-19",2022-08-10
3,4,4,6,7,Group Stage,None,2025-02-10
4,5,5,8,9,Final,"22-20, 21-19",2021-07-15


### Some Complex Queries

In [24]:
# Top 5 Players by Performance
query_player_performance = """
SELECT P.name, SUM(Pr.points_scored) AS Total_Points
FROM Players P
JOIN Performance Pr ON P.player_id = Pr.player_id
GROUP BY P.player_id
ORDER BY Total_Points DESC
LIMIT 5;
"""
player_performance = func_read_query(query_player_performance)
print("Top 5 Players by Performance:")
player_performance

Top 5 Players by Performance:


,name,Total_Points
0,Player283,311
1,Player286,249
2,Player196,225
3,Player330,222
4,Player68,222


In [25]:
# Most Successful Tournament Locations
query_tournament_location = """
SELECT T.location, COUNT(M.match_id) AS Matches_Held
FROM Tournaments T
JOIN Matches M ON T.tournament_id = M.tournament_id
GROUP BY T.location
ORDER BY Matches_Held DESC;
"""
tournament_location = func_read_query(query_tournament_location)
print("Most Successful Tournament Locations:")
tournament_location

Most Successful Tournament Locations:


,location,Matches_Held
0,Beijing,14
1,Tokyo,9
2,London,8
3,New York,7
4,Sydney,5
5,Los Angeles,4
6,Cape Town,4


In [26]:
# Sponsorship Distribution by Experience Level
query_sponsorship_distribution = """
SELECT P.experience_level, COUNT(S.sponsor_id) AS Sponsorships
FROM Players P
JOIN Sponsorships S ON P.player_id = S.player_id
GROUP BY P.experience_level;
"""
sponsorship_distribution = func_read_query(query_sponsorship_distribution)
print("Sponsorship Distribution by Experience Level:")
sponsorship_distribution

Sponsorship Distribution by Experience Level:


,experience_level,Sponsorships
0,None,2
1,Beginner,2
2,Intermediate,8
3,Professional,7


In [27]:
# Retrieve players with sponsorship details
query_sponsorship_details = """
SELECT P.name, S.sponsor_name, S.contract_value, S.duration_years
FROM Players P
JOIN Sponsorships S ON P.player_id = S.player_id
ORDER BY S.contract_value DESC
LIMIT 5;
"""
sponsorship_details = func_read_query(query_sponsorship_details)
print("Top 5 sponsorship deals:")
sponsorship_details

Top 5 sponsorship deals:


,name,sponsor_name,contract_value,duration_years
0,John Doe,Nike,500000.000000,3
1,Player3,Li-Ning,496544.444811,3
2,Emily Smith,Adidas,450000.000000,2
3,Wei Zhang,Victor,400000.000000,3
4,Player30,Yonex,372161.844661,4


In [28]:
# 3. Check performance stats for a specific player
query_performance_stats = """
SELECT P.name, M.match_id, Pr.points_scored, Pr.fouls, Pr.ranking_change
FROM Players P
JOIN Performance Pr ON P.player_id = Pr.player_id
JOIN Matches M ON Pr.match_id = M.match_id
ORDER BY Pr.points_scored DESC
LIMIT 5;
"""
performance_stats = func_read_query(query_performance_stats)

print("Top 5 player performances:")
performance_stats

Top 5 player performances:


,name,match_id,points_scored,fouls,ranking_change
0,Player354,170,100,0,-2
1,Player76,303,99,3,-5
2,Player113,245,99,5,4
3,Player348,167,99,3,5
4,Player167,223,99,2,-2


### Close Connection

In [29]:
# Commit changes and close connection
conn.commit()
conn.close()

### Download Database

In [30]:
from google.colab import files

files.download("sports_performance.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>